**HPO toolkit tutorial**

This notebook shows how to install and use HPO toolkit for work with Human Phenotype Ontology (HPO).

# Installation

The toolkit is available at [PyPi](https://pypi.org/project/hpo-toolkit), so installation with `pip` is really easy:

# API update

As of `v0.1.5`, we re-export most commonly used classes from the top-level package to save some typing:

|   Class          |      Previous                      |  New API (top-level reexport)   |
|:-----------------|:-----------------------------------|:--------------------------------|
| TermId           |  `hpotk.model.TermId`              | `hpotk.TermId`                  |
| Term             |  `hpotk.model.Term`                | `hpotk.Term`                    |
| MinimalTerm      |  `hpotk.model.MinimalTerm`         | `hpotk.MinimalTerm`             |
| Synonym          |  `hpotk.model.Synonym`             | `hpotk.Synonym`                 |
| SynonymType      |  `hpotk.model.SynonymType`         | `hpotk.SynonymType`             |
| SynonymCategory  |  `hpotk.model.SynonymCategory`     | `hpotk.SynonymCategory`         |
| OntologyGraph    |  `hpotk.graph.OntologyGraph`       | `hpotk.OntologyGraph`           |
| GraphAware       |  `hpotk.graph.GraphAware`          | `hpotk.GraphAware`              |
| Ontology         |  `hpotk.ontology.Ontology`         | `hpotk.Ontology`                |
| Ontology         |  `hpotk.ontology.Ontology`         | `hpotk.Ontology`                |
| MinimalOntology  |  `hpotk.ontology.MinimalOntology`  | `hpotk.MinimalOntology`         |


# Load HPO

HPO toolkit supports reading ontologies in [Obographs](https://github.com/geneontology/obographs) JSON format.

We can download and open the latest HPO from
> https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.json

In [1]:
from hpotk import Ontology
from hpotk.ontology.load.obographs import load_ontology

o: Ontology = load_ontology('http://purl.obolibrary.org/obo/hp.json')

We start by importing the relevant parts and use `load_ontolgy` to load the latest HPO in the Obographs format. We get an instance of `Ontology` which is a container for ontology data.

Let's see how HPO toolkit models the ontology data.

# Ontology data model and functionality

HPO toolkit includes several classes that serve as building blocks in the data model. This section provides basic information, starting from the bottom of the class hierarchy.

- `hpotk.TermId` - an identifier of an ontology concept.
- `hpotk.MinimalTerm` - represents minimal useful information of the ontology concept. `MinimalTerm` has the following attributes:
  - `identifier`, `TermId` (e.g. `HP:0001166`)
  - `name`, `str` (e.g. `Arachnodactyly`)
  - `is_current`/`is_obsolete`, whether or not the concept has been obsoleted
  - `alt_term_ids`, a sequence of obsolete `TermId`s that represented the term previously
- `hpotk.Term` - the complete info regarding the ontology concept. The `Term` has all attributes of the `MinimalTerm` plus the following:
  - `definition` - an optional description of the term in slightly more verbiage
  - `comment` - additional comment (optional)
  - `synonyms` - alternative designations of the `Term` (optional)
  - `xrefs` - a sequence of cross-references between the `Term` and concepts from different databases
- `hpotk.MinimalOntology` - the container for ontology data that uses `MinimalTerm`s
- `hpotk.Ontology` - the ontology data container that contains `Term`s
- `hpotk.OntologyGraph` - a specification of graph for storing the ontology concept hierarchy and the required graph functionality. As long as the graph implements the methods, it can work with the rest of the toolkit framework

Now, let's go over some examples to explore the provided functionality.

## Get all `Term`s and `TermId`s

All `TermId`s, both *primary* and *obsolete* can be iterated over via `o.term_ids` property:

In [2]:
print(next(iter(o.term_ids)))

HP:0000001


Similarly, we can iterate over ontology `Term`s via `hpo.terms`:

In [3]:
print(next(iter(o.terms)))

Term(identifier=HP:0000001, name="All", definition=None, comment=Root of all terms in the Human Phenotype Ontology., synonyms=None, xrefs=[DefaultTermId(idx=4, value=UMLS:C0444868)], is_obsolete=False, alt_term_ids="[]")


and get the number of the primary (non-obsolete) `Term`s:

In [4]:
len(o)

17138

## Query `Term`

### Test presence of a `TermId` in the ontology

We can test presence of a `TermId` in the same way as we test presence of an item in a Python container:

In [5]:
current_arachnodactyly_id = "HP:0001166"  # as of Dec 28th, 2022

assert current_arachnodactyly_id in o

The test works for both *primary* and *obsolete* `TermId`s:

In [6]:
obsolete_arachnodactyly_id = "HP:0001505"

assert obsolete_arachnodactyly_id in o

Queries work with a simple CURIE `str` (e.g. `HP:0001166`) or a `TermId`:

In [7]:
from hpotk import TermId
assert current_arachnodactyly_id in o and TermId.from_curie(current_arachnodactyly_id) in o

### Get a specific `Term`

We can get ahold of a specific `Term` using the `get_term` method:

In [8]:
arachnodactyly = o.get_term(current_arachnodactyly_id)
print(arachnodactyly)

Term(identifier=HP:0001166, name="Arachnodactyly", definition=Abnormally long and slender fingers ("spider fingers")., comment=None, synonyms=[Synonym(name="Long slender fingers", category=SynonymCategory.EXACT, synonym_type=SynonymType.LAYPERSON_TERM, xrefs="None"), Synonym(name="Long, slender fingers", category=SynonymCategory.EXACT, synonym_type=None, xrefs="None"), Synonym(name="Spider fingers", category=SynonymCategory.EXACT, synonym_type=SynonymType.LAYPERSON_TERM, xrefs="None")], xrefs=[DefaultTermId(idx=3, value=MSH:D054119), DefaultTermId(idx=11, value=SNOMEDCT_US:62250003), DefaultTermId(idx=4, value=UMLS:C0003706)], is_obsolete=False, alt_term_ids="[DefaultTermId(idx=2, value=HP:0001505)]")


`get_term` always returns the primary `Term`, even for an obsolete `TermId`:

In [9]:
assert o.get_term(current_arachnodactyly_id) == o.get_term(obsolete_arachnodactyly_id)

# Ontology algorithms

The `hpotk.algorithm` module provides several ontology algorithms.

## Ontology traversal

Use `get_parents` and `get_ancestors` to get a `set` with *parents* or *ancestors* of a `TermId`:

In [10]:
import hpotk

print('#' * 20 + ' Parents ' + '#' * 20)
for parent in hpotk.algorithm.get_parents(o, current_arachnodactyly_id):
    term = o.get_term(parent)
    print(f"{term.identifier.value} - {term.name}")

print('\n'+'#' * 20 + ' Ancestors ' + '#' * 18)
for parent in hpotk.algorithm.get_ancestors(o, current_arachnodactyly_id):
    term = o.get_term(parent)
    print(f"{term.identifier.value} - {term.name}")

#################### Parents ####################
HP:0100807 - Long fingers
HP:0001238 - Slender finger

#################### Ancestors ##################
HP:0000924 - Abnormality of the skeletal system
HP:0000001 - All
HP:0040064 - Abnormality of limbs
HP:0002817 - Abnormality of the upper limb
HP:0040068 - Abnormality of limb bone
HP:0001167 - Abnormal finger morphology
HP:0033127 - Abnormality of the musculoskeletal system
HP:0100807 - Long fingers
HP:0011844 - Abnormal appendicular skeleton morphology
HP:0001238 - Slender finger
HP:0002813 - Abnormality of limb bone morphology
HP:0000118 - Phenotypic abnormality
HP:0001155 - Abnormality of the hand
HP:0011297 - Abnormal digit morphology
HP:0011842 - Abnormal skeletal morphology


We can get the *children* or *descendants* by calling `get_children` and `get_descendants`:

In [11]:
print('#' * 20 + ' Children ' + '#' * 20)
for child in hpotk.algorithm.get_children(o, 'HP:0100807'):  # Long fingers
    term = o.get_term(child)
    print(f"{term.identifier.value} - {term.name}")
    
print('\n'+'#' * 20 + ' Descendants ' + '#' * 17)
for descendant in hpotk.algorithm.get_descendants(o, 'HP:0006109'):  # Absent phalangeal crease 
    term = o.get_term(descendant)
    print(f"{term.identifier.value} - {term.name}")

#################### Children ####################
HP:0001182 - Tapered finger
HP:0001166 - Arachnodactyly

#################### Descendants #################
HP:0006216 - Single interphalangeal crease of fifth finger
HP:0006077 - Absent proximal finger flexion creases
HP:0001032 - Absent distal interphalangeal creases
HP:0005780 - Absent fourth finger distal interphalangeal crease


# Validation

The toolkit provides functions for performing multiple useful sanity checks.

## Obsolete term IDs

We should always use the primary term IDs instead of the obsolete terms.

`HP:0100807` and `HP:0006010` are the primary and obsolete term IDs for *Long fingers*. 
The `ObsoleteTermIdsValidator` points out usage of the obsolete term:

In [12]:
from hpotk import MinimalTerm

from hpotk.validate import ValidationLevel
from hpotk.validate import ObsoleteTermIdsValidator

obso_validator = ObsoleteTermIdsValidator(o)

# The term uses an obsolete term ID `HP:0006010` instead of the current `HP:0100807`.
inputs = [
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0006010'), name='Long fingers', alt_term_ids=[], is_obsolete=False)
]
results = obso_validator.validate(inputs)

# At least one error or warning was found
assert results.is_ok() == False

# A sequence of errors/warnings is availabe via `results` property
assert len(results.results) == 1

validation_result = results.results[0]

# Obsolete term ID is a warning. The toolkit can map the term ID to the primary term ID and use it in the downstream analyses
assert validation_result.level == ValidationLevel.WARNING

# A unique ID of the validation check
assert validation_result.category == 'obsolete_term_id_is_used'

# A message for human consumption
assert validation_result.message == 'Using the obsolete HP:0006010 instead of HP:0100807 for Long fingers'

## Violation of the annotation propagation rule

A set of HPO terms should not contain both term and its ancestor because the annotation propagation rule implies presence of all term's ancestors.

In [13]:
from hpotk.validate import AnnotationPropagationValidator

# Long fingers is a parent of Arachnodactyly
inputs = [
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0001166'), name='Arachnodactyly', alt_term_ids=[], is_obsolete=False),
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0100807'), name='Long fingers', alt_term_ids=[], is_obsolete=False)
]

ap_validator = AnnotationPropagationValidator(o)
results = ap_validator.validate(inputs)

val_result = results.results[0]

# Violation of the annotation propagation rule is an ERROR.
# Most analyses will yield biased/incorrect results when using both the term and its ancestor.
assert val_result.level == ValidationLevel.ERROR

# A unique ID of the validation check
assert val_result.category == 'annotation_propagation'

# A message for human consumption
assert val_result.message == 'Terms should not contain both Arachnodactyly [HP:0001166] and its ancestor Long fingers [HP:0100807]'

## Terms are phenotypic features

Most algorithms require a list of phenotypic features (i.e. descendants of [Phenotypic abnormality](https://hpo.jax.org/app/browse/term/HP:0000118)) 
but HPO contains terms that are does not represent phenotypic abnormality, such as [Clinical modifier](https://hpo.jax.org/app/browse/term/HP:0012823), [Mode of inheritance](https://hpo.jax.org/app/browse/term/HP:0000005), etc.

`PhenotypicAbnormalityValidator` reports all terms that are not descendants of [Phenotypic abnormality](https://hpo.jax.org/app/browse/term/HP:0000118).

In [14]:
from hpotk.validate import PhenotypicAbnormalityValidator

# Long fingers is a parent of Arachnodactyly
inputs = [
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0000007'), name='Autosomal recessive inheritance', alt_term_ids=[], is_obsolete=False),
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0100807'), name='Long fingers', alt_term_ids=[], is_obsolete=False)
]

pa_validator = PhenotypicAbnormalityValidator(o)
results = pa_validator.validate(inputs)

val_result = results.results[0]

# Using non-phenotypic abnormality is an ERROR.
assert val_result.level == ValidationLevel.ERROR

# A unique ID of the validation check
assert val_result.category == 'phenotypic_abnormality_descendant'

# A message for human consumption
assert val_result.message == 'Autosomal recessive inheritance [HP:0000007] is not a descendant of Phenotypic abnormality [HP:0000118]'

## Perform multiple checks at the same time

The checks can be performed individually, as described in the previous sections, or together within a single validation run using `ValidationRunner`.

In [15]:
from hpotk.validate import ValidationRunner

inputs = [
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0000007'), name='Autosomal recessive inheritance', alt_term_ids=[], is_obsolete=False),
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0006010'), name='Long fingers', alt_term_ids=[], is_obsolete=False),
    MinimalTerm.create_minimal_term(TermId.from_curie('HP:0001166'), name='Arachnodactyly', alt_term_ids=[], is_obsolete=False),
]

runner = ValidationRunner(validators=[obso_validator, ap_validator, pa_validator])
results = runner.validate_all(inputs)
for issue in results.results:
    print(issue.message)

Using the obsolete HP:0006010 instead of HP:0100807 for Long fingers
Terms should not contain both Arachnodactyly [HP:0001166] and its ancestor Long fingers [HP:0100807]
Autosomal recessive inheritance [HP:0000007] is not a descendant of Phenotypic abnormality [HP:0000118]


# HPO constants

The toolkit provides `TermId`s of the well-established and stable HPO concepts.

## Base terms

The terms that define the first level of HPO hierarchy (e.g. [Phenotypic abnormality](https://hpo.jax.org/app/browse/term/HP:0000118)):

In [16]:
from hpotk.constants.hpo.base import PHENOTYPIC_ABNORMALITY
print(PHENOTYPIC_ABNORMALITY)

HP:0000118


## Frequency

Concepts to represent frequency of phenotypic abnormalities within a patient cohort, the children of [Frequency](https://hpo.jax.org/app/browse/term/HP:0040279) (e.g. [Occasional](https://hpo.jax.org/app/browse/term/HP:0040283)):

In [17]:
from hpotk.constants.hpo.frequency import OCCASIONAL
print(OCCASIONAL)

HP:0040283


On top of the frequency `TermId` constants, the toolkit provides the `HpoFrequency` class that groups frequency `TermId`s with the frequency bounds:

In [18]:
from hpotk.constants.hpo.frequency import parse_hpo_frequency

parse_hpo_frequency(OCCASIONAL)

HpoFrequency(identifier=HP:0040283, lower_bound=0.05, upper_bound=0.29)

The lookup works both for a `str` CURIE and a `TermId`:

In [19]:
occasional_id = 'HP:0040283'
assert parse_hpo_frequency(OCCASIONAL) == parse_hpo_frequency(occasional_id)

## Inheritance

Selected descendents of [Mode of inheritance](https://hpo.jax.org/app/browse/term/HP:0000005) (e.g. [Autosomal dominant inheritance](https://hpo.jax.org/app/browse/term/HP:0000006)):

In [20]:
from hpotk.constants.hpo.inheritance import AUTOSOMAL_DOMINANT_INHERITANCE
print(AUTOSOMAL_DOMINANT_INHERITANCE)

HP:0000006


## Onset

All descendents of [Onset](https://hpo.jax.org/app/browse/term/HP:0003674) (e.g. [Congenital onset](https://hpo.jax.org/app/browse/term/HP:0003577)):

In [21]:
from hpotk.constants.hpo.onset import CONGENITAL_ONSET
print(CONGENITAL_ONSET)

HP:0003577


## Organ system

Children of [Phenotypic abnormality](https://hpo.jax.org/app/browse/term/HP:0000118) that correspond to abnormalities of organ systems (e.g. [Abnormality of the ear](https://hpo.jax.org/app/browse/term/HP:0000598)):

In [22]:
from hpotk.constants.hpo.organ_system import ABNORMALITY_OF_THE_EAR
print(ABNORMALITY_OF_THE_EAR)

HP:0000598


## Severity

All descendents of [Severity](https://hpo.jax.org/app/browse/term/HP:0012824) (e.g. [Mild](https://hpo.jax.org/app/browse/term/HP:0012825)):

In [23]:
from hpotk.constants.hpo.severity import MILD
print(MILD)

HP:0012825


# HPO annotations

HPO toolkit provides a data model and IO for working with Mendelian diseases.

The data model consists of the following classes:
- `hpotk.annotations.HpoDiseaseAnnotation` - represents data available for a phenotypic feature of a disease, such as the feature identifier (`TermId`), frequency of occurrence (`hpotk.annotations.Ratio`), references (a list of `hpotk.annotations.AnnotationReference`s), and modifiers (a list of `TermId`s that should be the descendants of the [Clinical modifier](https://hpo.jax.org/app/browse/term/HP:0012823) HPO concept).
- `hpotk.annotations.HpoDisease` - the disease model that has an identifier (`TermId`), name (`str`), phenotypic features/annotations (a collection of `HpoDiseaseAnnotation`s), and modes of inheritance (a collection of `TermId`s from the [Mode of inheritance](https://hpo.jax.org/app/browse/term/HP:0000005) HPO sub-hierarchy)

## Load HPO annotations

The toolkit provides parser for reading HPO annotation file format. Get the latest annotation from the [HPO download section](https://hpo.jax.org/app/data/annotations):
> http://purl.obolibrary.org/obo/hp/hpoa/phenotype.hpoa

In [24]:
from hpotk.annotations import HpoDiseases
from hpotk.annotations.load.hpoa import SimpleHpoaDiseaseLoader

loader = SimpleHpoaDiseaseLoader(o)
diseases: HpoDiseases = loader.load('http://purl.obolibrary.org/obo/hp/hpoa/phenotype.hpoa')
f'Parsed {len(diseases)} diseases'

'Parsed 12429 diseases'

The loader parser the HPO annotation file into `HpoDiseases`; a container of disease models. 

The container supports iteration over diseases,

In [25]:
print(next(iter(diseases.items)))

HpoDisease(identifier=OMIM:619340, name=Developmental and epileptic encephalopathy 96, n_annotations=9)


iteration over disease IDs,

In [26]:
# Iterate over disease identifiers
print(next(iter(diseases.item_ids())))

OMIM:619340


and lookup of a disease by ID (either a CURIE `str` or a `TermId`)

In [27]:
# Look up the disease by ID
print(diseases['OMIM:154700'])

HpoDisease(identifier=OMIM:154700, name=Marfan syndrome, n_annotations=69)


Each `HpoDisease` has the following attributes:
- `identifier` - `TermId` corresponding to the identifier
- `name` - disease name (e.g. *Marfan syndrome*)
- `annotations` - a collection of disease annotations
- `modes_of_inheritance` - a collection of modes of inheritance

In [28]:
marfan = diseases['OMIM:154700']
print(f'ID: {marfan.identifier}')
print(f'Name: {marfan.name}')
marfan_features = [o.get_term(a.identifier).name for a in marfan.annotations]
print(f'The annotation count: {len(marfan.annotations)}')
print(f'The first 5 annotations: {", ".join(marfan_features[:5])}')
mois = [o.get_term(moi).name for moi in marfan.modes_of_inheritance]
print(f'Modes of inheritance: {mois}')

ID: OMIM:154700
Name: Marfan syndrome
The annotation count: 69
The first 5 annotations: Astigmatism, Limited elbow extension, Strabismus, Mitral annular calcification, Flexion contracture
Modes of inheritance: ['Autosomal dominant inheritance']


The HPO annotations API is still evolving. Stay tuned for more features to come!

# Semantic similarity

## Information content of ontology terms

We can calculate information content (IC) for all ontology terms and obtain `AnnotationIcContainer`:

In [29]:
from hpotk.algorithm.similarity import calculate_ic_for_annotated_items, AnnotationIcContainer

term_id2ic: AnnotationIcContainer = calculate_ic_for_annotated_items(diseases, o)

`AnnotationIcContainer` is a mapping from `TermId` to IC (`float`):

In [30]:
ic_arachnodactyly = term_id2ic[arachnodactyly.identifier]
print(f'IC of {arachnodactyly.name}: {ic_arachnodactyly} nats')

IC of Arachnodactyly: 7.259769077238713 nats


By default, the `base` parameter of the `calculate_ic_for_annotated_items` is set to $e$ (Euler's number), returning the IC values in [nats](https://en.wikipedia.org/wiki/Nat_(unit)). Set `base=2` to get the IC in bits.

The versions of the items and ontology used to calculate the ICs are preserved in the `metadata` attribute of the `AnnotationIcContainer`:

In [31]:
term_id2ic.metadata

{'annotated_items_version': '2023-04-05', 'ontology_version': '2023-04-06'}

## Resnik similarity

We can use the term IC to calculate semantic similarity of items (e.g. diseases, patients) annotated with ontology concepts.

Resnik similarity uses the information content of the *most informative common ancestor* ($IC_{MICA}$ to calculate semantic similarity. `hpo-toolkit` offers a function for calculating $IC_{MICA}$ for all ontology concept pairs:

In [32]:
from hpotk.algorithm.similarity import precalculate_ic_mica_for_hpo_concept_pairs, SimilarityContainer

#sc: SimilarityContainer = precalculate_ic_mica_for_hpo_concept_pairs(term_id2ic, o)

The function is commented out/not run in the notebook because it is a bit slow..

However, for the purpose of the demonstration, let's assume we have the `SimilarityContainer` on hand.

In [33]:
sc = SimilarityContainer()

We can query the $IC_{MICA}$ of two HPO terms by:

In [34]:
a = 'HP:0001166'  # Arachnodactyly
b = 'HP:0001182'  # Tapered finger
ic_mica = sc.get_similarity(a, b)

That's it for now!